In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import joblib
from datetime import datetime
import os
import re
import datetime

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fetch NHL odds data
url = 'https://sports.yahoo.com/nhl/odds/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract tables from the webpage
tables = soup.find_all('table')
data = []

for table in tables:
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append(cols)

# Convert table data to DataFrame
df = pd.DataFrame(data)

# Filter rows where the first column length is greater than 5
filtered_df = df[df[0].astype(str).map(len) > 5]

# Extract team names and odds
filtered_df['Team'] = filtered_df[0].apply(lambda x: x.split('(')[0].strip())
filtered_df['Odds'] = filtered_df[1].apply(lambda x: x[-4:].strip())

# Create a new DataFrame with relevant columns
new_df = filtered_df[['Team', 'Odds']]

# Load team abbreviations and create mapping dictionary
abbreviations_df = pd.read_csv('teams.csv')

# Clean team names in abbreviations_df (strip spaces but keep original capitalization)
#def clean_team_name(name):
    #return name.strip()

#abbreviations_df['Team'] = abbreviations_df['Team'].apply(clean_team_name)
mapping_dict = dict(zip(abbreviations_df['Name'], abbreviations_df['Abbrv']))

# Clean team names in filtered_df
#filtered_df['Team'] = filtered_df['Team'].apply(clean_team_name)

# Map team names to abbreviations
def map_team_name(team):
    if team in mapping_dict:
        return mapping_dict[team]
    else:
        print(f"Unmapped team: {team}")  # Log missing teams
        return team  # Optionally keep original name for unmapped teams

new_df['Team'] = filtered_df['Team'].apply(map_team_name)

# Convert American odds to decimal odds
def convert_american_to_decimal(american_odds):
    if american_odds.startswith('+'):
        return 1 + int(american_odds[1:]) / 100
    elif american_odds.startswith('-'):
        return 1 + 100 / abs(int(american_odds))
    else:
        return None

new_df['Decimal Odds'] = new_df['Odds'].apply(convert_american_to_decimal)
new_df['Decimal Odds'] = new_df['Decimal Odds'].round(2)

# Convert decimal odds to implied probability
def decimal_to_implied_probability(decimal_odds):
    if decimal_odds > 0:
        return round(1 / decimal_odds, 2)
    else:
        return None

new_df['Implied Probability'] = new_df['Decimal Odds'].apply(decimal_to_implied_probability)

# Save the final DataFrame to a CSV file
new_df.to_csv('nhl_odds.csv', index=False)

print("Script completed. Check 'nhl_odds.csv' for the results.")


Script completed. Check 'nhl_odds.csv' for the results.


C:\Users\luken\AppData\Local\Temp\ipykernel_17076\1022136405.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Team'] = filtered_df[0].apply(lambda x: x.split('(')[0].strip())
C:\Users\luken\AppData\Local\Temp\ipykernel_17076\1022136405.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Odds'] = filtered_df[1].apply(lambda x: x[-4:].strip())
C:\Users\luken\AppData\Local\Temp\ipykernel_17076\1022136405.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s